<a href="https://colab.research.google.com/github/jsaulme/Datos_Masivos_2024/blob/main/DatosMasivos_Tarea1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalacion de Spark

In [ ]:
pip install pyspark

In [ ]:
!pip install spotipy

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
50 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as

In [ ]:
import os
import sys
import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.10/dist-packages/pyspark'

In [ ]:
import pyspark
from pyspark.sql import SparkSession, DataFrame
from typing import List
import pyspark.sql.functions as F
from pyspark.sql.types import T

In [ ]:
spark = SparkSession.builder.appName("Primera Clase de Spark").getOrCreate()
spark

# Importar librerias necesarias

In [ ]:
import statistics
import requests
import base64
import client_credential  # Asegúrate de que este archivo contiene CLIENT_SECRET de forma segura.
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
# Configuración de credenciales de Spotify desde variables de entorno o archivo seguro
CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID", "d73dbd2a7d1248b8bc326eec8586a0e3")
CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET", client_credential.CLIENT_SECRET)

def get_access_token(client_id: str, client_secret: str) -> str:
    """Obtiene un token de acceso utilizando client_credentials."""
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + base64.b64encode(f"{client_id}:{client_secret}".encode()).decode(),
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials"
    }

    response = requests.post(url, headers=headers, data=data)
    if response.status_code != 200:
        error_msg = response.json().get("error_description", "No se pudo obtener el token de acceso")
        raise Exception(f"Error al obtener el token de acceso: {error_msg}")

    return response.json()["access_token"]

def create_spotify_client(access_token: str) -> spotipy.Spotify:
    """Crea una instancia del cliente de Spotify con el token de acceso."""
    credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
    sp = spotipy.Spotify(client_credentials_manager=credentials_manager)
    return sp

def search_artist(token: str, artist_name: str) -> str:
    """Busca el ID de un artista dado su nombre."""
    sp = spotipy.Spotify(auth=token)
    results = sp.search(q=artist_name, type="artist", limit=1)

    if results["artists"]["items"]:
        return results["artists"]["items"][0]["id"]
    else:
        raise Exception(f"No se encontró al artista '{artist_name}'.")

def get_artist_data(token: str, artist_id: str) -> dict:
    """Obtiene datos del artista como seguidores y popularidad."""
    sp = spotipy.Spotify(auth=token)
    artist = sp.artist(artist_id)
    return {
        "name": artist["name"],
        "followers": artist["followers"]["total"],
        "popularity": artist["popularity"],
        "genres": artist["genres"]
    }

def get_artist_top_tracks(token: str, artist_id: str) -> list:
    """Obtiene las canciones más populares del artista en todos los mercados disponibles."""
    sp = spotipy.Spotify(auth=token)
    top_tracks = sp.artist_top_tracks(artist_id)

    # Extrae las pistas con sus mercados disponibles
    return [
        {
            "track_name": track["name"],
            "popularity": track["popularity"],
            "album": track["album"]["name"],
            "available_markets": track["available_markets"]  # Listado de mercados por track
        }
        for track in top_tracks["tracks"]
    ]

def compare_artists(token: str, artist_name1: str, artist_name2: str) -> None:
    """Realiza una comparativa entre dos artistas, mostrando datos clave, incluyendo todos los mercados disponibles."""
    # Buscar IDs de los artistas
    artist_id1 = search_artist(token, artist_name1)
    artist_id2 = search_artist(token, artist_name2)
    print(f'El identificador del artista {artist_name1} es: {artist_id1}')
    print(f'El identificador del artista {artist_name2} es: {artist_id2}')

    # Obtener datos de los artistas
    artist_data1 = get_artist_data(token, artist_id1)
    artist_data2 = get_artist_data(token, artist_id2)
    print(f"\nDatos del artista {artist_name1}: {artist_data1}")
    print(f"Datos del artista {artist_name2}: {artist_data2}")

    # Obtener las canciones más populares de cada artista
    artist1_top_tracks = get_artist_top_tracks(token, artist_id1)
    artist2_top_tracks = get_artist_top_tracks(token, artist_id2)

    print(f"\nCanciones más populares de {artist_name1}:")
    for track in artist1_top_tracks:
        print(f"{track['track_name']} (Popularidad: {track['popularity']}) - Mercados: {', '.join(track['available_markets'])}")

    print(f"\nCanciones más populares de {artist_name2}:")
    for track in artist2_top_tracks:
        print(f"{track['track_name']} (Popularidad: {track['popularity']}) - Mercados: {', '.join(track['available_markets'])}")


In [ ]:
def compare_metrics(artist_data1: dict, artist_data2: dict, artist1_top_tracks: list, artist2_top_tracks: list) -> None:
    """Compara métricas clave de dos artistas y determina el ganador."""

    # Inicializar contadores de puntuación
    artist1_score, artist2_score = 0, 0

    # Verificar que artist_data1 y artist_data2 sean diccionarios válidos
    if not isinstance(artist_data1, dict) or not isinstance(artist_data2, dict):
        print("Error: Datos de artistas no válidos.")
        print(f"artist_data1: {artist_data1}, artist_data2: {artist_data2}")
        return

    # Comparativa de Seguidores
    print("\n--- Comparativa de Seguidores ---")
    followers1 = artist_data1.get("followers", 0)
    followers2 = artist_data2.get("followers", 0)
    print(f"Seguidores de {artist_data1.get('name', 'Artista 1')}: {followers1}")
    print(f"Seguidores de {artist_data2.get('name', 'Artista 2')}: {followers2}")

    if followers1 > followers2:
      print(f"Ganador en Seguidores: {artist_data1.get('name', 'Artista 1')}")
      artist1_score += 1
    elif followers1 < followers2:
      print(f"Ganador en Seguidores: {artist_data2.get('name', 'Artista 2')}")
      artist2_score += 1
    else:
      print("Empate en Seguidores")

    # Comparativa de Popularidad
    print("\n--- Comparativa de Popularidad ---")
    popularity1 = artist_data1.get("popularity", 0)
    popularity2 = artist_data2.get("popularity", 0)
    print(f"Popularidad de {artist_data1.get('name', 'Artista 1')}: {popularity1}")
    print(f"Popularidad de {artist_data2.get('name', 'Artista 2')}: {popularity2}")

    if popularity1 > popularity2:
        print(f"Ganador en Popularidad: {artist_data1.get('name', 'Artista 1')}")
        artist1_score += 1
    elif popularity1 < popularity2:
        print(f"Ganador en Popularidad: {artist_data2.get('name', 'Artista 2')}")
        artist2_score += 1
    else:
        print("Empate en Popularidad")

    # Comparativa de Mercados Disponibles
    print("\n--- Comparativa de Mercados Disponibles ---")
    artist1_markets = set()
    artist2_markets = set()

    for track in artist1_top_tracks:
        if isinstance(track, dict):
            artist1_markets.update(track.get("available_markets", []))
    for track in artist2_top_tracks:
        if isinstance(track, dict):
            artist2_markets.update(track.get("available_markets", []))
    print(f"Mercados disponibles para {artist_data1.get('name', 'Artista 1')}: {len(artist1_markets)}")
    print(f"Mercados disponibles para {artist_data2.get('name', 'Artista 2')}: {len(artist2_markets)}")

    if len(artist1_markets) > len(artist2_markets):
        print(f"Ganador en Mercados Disponibles: {artist_data1.get('name', 'Artista 1')}")
        artist1_score += 1
    elif len(artist1_markets) < len(artist2_markets):
        print(f"Ganador en Mercados Disponibles: {artist_data2.get('name', 'Artista 2')}")
        artist2_score += 1
    else:
        print("Empate en Mercados Disponibles")

    import statistics

    # Comparativa de Canciones Populares
    print("\n--- Comparativa de Canciones Populares ---")

    # Obtener listas de popularidad de las canciones de cada artista
    artist1_popularities = [track.get("popularity", 0) for track in artist1_top_tracks if isinstance(track, dict)]
    artist2_popularities = [track.get("popularity", 0) for track in artist2_top_tracks if isinstance(track, dict)]

    # Mostrar número de canciones populares
    print(f"Número de canciones populares de {artist_data1.get('name', 'Artista 1')}: {len(artist1_popularities)}")
    print(f"Número de canciones populares de {artist_data2.get('name', 'Artista 2')}: {len(artist2_popularities)}")

    # Calcular la media y la mediana de popularidad
    mean_popularity1 = statistics.mean(artist1_popularities) if artist1_popularities else 0
    mean_popularity2 = statistics.mean(artist2_popularities) if artist2_popularities else 0
    median_popularity1 = statistics.median(artist1_popularities) if artist1_popularities else 0
    median_popularity2 = statistics.median(artist2_popularities) if artist2_popularities else 0

    print(f"Media de popularidad de {artist_data1.get('name', 'Artista 1')}: {mean_popularity1}")
    print(f"Media de popularidad de {artist_data2.get('name', 'Artista 2')}: {mean_popularity2}")
    print(f"Mediana de popularidad de {artist_data1.get('name', 'Artista 1')}: {median_popularity1}")
    print(f"Mediana de popularidad de {artist_data2.get('name', 'Artista 2')}: {median_popularity2}")

    # Determinar el ganador en popularidad de canciones
    if mean_popularity1 > mean_popularity2 and median_popularity1 > median_popularity2:
        print(f"Ganador en Popularidad de Canciones: {artist_data1.get('name', 'Artista 1')}")
        artist1_score += 1
    elif mean_popularity1 < mean_popularity2 and median_popularity1 < median_popularity2:
        print(f"Ganador en Popularidad de Canciones: {artist_data2.get('name', 'Artista 2')}")
        artist2_score += 1
    else:
        print("Empate en Popularidad de Canciones")

    # Devolver el puntaje actualizado
    return artist1_score, artist2_score


    # Ganador final
    if artist1_score > artist2_score:
      print(f"\nEl ganador general es: {artist_data1.get('name', 'Artista 1')} con {artist1_score} puntos")
    elif artist1_score < artist2_score:
      print(f"\nEl ganador general es: {artist_data2.get('name', 'Artista 2')} con {artist2_score} puntos")
    else:
      print("\nEs un empate entre ambos artistas.")

# Ejecutar la comparativa con datos verificados
compare_metrics(artist_data1, artist_data2, artist1_top_tracks, artist2_top_tracks)



--- Comparativa de Seguidores ---
Seguidores de Intocable: 3966116
Seguidores de Duelo: 1958004
Ganador en Seguidores: Intocable

--- Comparativa de Popularidad ---
Popularidad de Intocable: 73
Popularidad de Duelo: 72
Ganador en Popularidad: Intocable

--- Comparativa de Mercados Disponibles ---
Mercados disponibles para Intocable: 183
Mercados disponibles para Duelo: 185
Ganador en Mercados Disponibles: Duelo

--- Comparativa de Canciones Populares ---
Número de canciones populares de Intocable: 10
Número de canciones populares de Duelo: 10
Media de popularidad de Intocable: 64.4
Media de popularidad de Duelo: 64
Mediana de popularidad de Intocable: 64.5
Mediana de popularidad de Duelo: 64.0
Ganador en Popularidad de Canciones: Intocable


(3, 1)

In [ ]:
if __name__ == "__main__":
    try:
        # Obtener el token de acceso
        token = get_access_token(CLIENT_ID, CLIENT_SECRET)
        print("Token de acceso obtenido correctamente.")

        # Definir los nombres de los artistas
        artist_name1 = "Intocable"
        artist_name2 = "Duelo"

        # Buscar los IDs de los artistas
        artist_id1 = search_artist(token, artist_name1)
        artist_id2 = search_artist(token, artist_name2)

        # Obtener datos de los artistas
        artist_data1 = get_artist_data(token, artist_id1)
        artist_data2 = get_artist_data(token, artist_id2)

        # Obtener las canciones más populares de cada artista
        artist1_top_tracks = get_artist_top_tracks(token, artist_id1)
        artist2_top_tracks = get_artist_top_tracks(token, artist_id2)

        # Realizar la comparativa de métricas
        compare_metrics(artist_data1, artist_data2, artist1_top_tracks, artist2_top_tracks)

    except Exception as e:
        print(f"Error: {e}")


Token de acceso obtenido correctamente.

--- Comparativa de Seguidores ---
Seguidores de Intocable: 3966116
Seguidores de Duelo: 1958004
Ganador en Seguidores: Intocable

--- Comparativa de Popularidad ---
Popularidad de Intocable: 73
Popularidad de Duelo: 72
Ganador en Popularidad: Intocable

--- Comparativa de Mercados Disponibles ---
Mercados disponibles para Intocable: 183
Mercados disponibles para Duelo: 185
Ganador en Mercados Disponibles: Duelo

--- Comparativa de Canciones Populares ---
Número de canciones populares de Intocable: 10
Número de canciones populares de Duelo: 10
Media de popularidad de Intocable: 64.4
Media de popularidad de Duelo: 64
Mediana de popularidad de Intocable: 64.5
Mediana de popularidad de Duelo: 64.0
Ganador en Popularidad de Canciones: Intocable


In [ ]:
    # Imprimir datos completos de los artistas para verificar su contenido
    print("\n--- Datos de Artista 1 ---")
    print(artist_data1)
    print("\n--- Datos de Artista 2 ---")
    print(artist_data2)


--- Datos de Artista 1 ---
{'name': 'Intocable', 'followers': 3966116, 'popularity': 73, 'genres': ['banda', 'musica mexicana', 'norteno', 'tejano']}

--- Datos de Artista 2 ---
{'name': 'Duelo', 'followers': 1958004, 'popularity': 72, 'genres': ['banda', 'musica mexicana', 'norteno', 'tejano']}


In [ ]:
# Obtener el identificador de los artistas
artist_id1 = search_artist(token, artist_name1)
artist_id2 = search_artist(token, artist_name2)
print(f'El identificador del artista {artist_name1} es: {artist_id1}')
print(f'El identificador del artista {artist_name2} es: {artist_id2}')

El identificador del artista Intocable es: 108moq3rq6bm1M4Ypz0J02
El identificador del artista Duelo es: 0nnp7oJpY2J6yZOqtdKaWq


In [ ]:
import requests
import pandas as pd

# Datos de autenticación (reemplaza con tus credenciales)
CLIENT_ID = 'TU_CLIENT_ID'
CLIENT_SECRET = 'TU_CLIENT_SECRET'

# Obtener información de álbumes de un artista
def get_artist_albums(artist_id, access_token):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/albums?include_groups=album,single"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        return response.json().get('items', [])
    else:
        print(f"Error fetching albums: {response.status_code}, {response.text}")
        return []  # Return an empty list to avoid further errors

# Obtener información de pistas (tracks) de un álbum
def get_album_tracks(album_id, access_token):
    url = f"https://api.spotify.com/v1/albums/{album_id}/tracks"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json().get('items', [])
    else:
        print(f"Error fetching tracks: {response.status_code}, {response.text}")
        return []

# Obtener listas de reproducción (playlists) del artista
def get_artist_playlists(artist_name, access_token):
    url = f"https://api.spotify.com/v1/search?q={artist_name}&type=playlist&limit=10"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.get(url, headers=headers)

    # Manejo de errores
    if response.status_code == 200:
        return response.json().get('playlists', {}).get('items', [])
    else:
        print(f"Error al obtener listas de reproducción: {response.status_code}, {response.text}")
        return []

# Obtener géneros
def get_genres(access_token):
    url = "https://api.spotify.com/v1/recommendations/available-genre-seeds"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json().get('genres', [])
    else:
        print(f"Error al obtener géneros: {response.status_code}, {response.text}")
        return []

# Obtener mercados (market)
def get_markets(access_token):
    url = "https://api.spotify.com/v1/markets"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json().get('markets', [])
    else:
        print(f"Error al obtener mercados: {response.status_code}, {response.text}")
        return []

# Guardar todos los datos en un archivo CSV
def save_data_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

# Código principal
if __name__ == "__main__":
    artist_id = "0bR8CYYg0vU4F5gJb79X5V"  # ID de Intocable

    # Obtener información de álbumes
    albums = get_artist_albums(artist_id, access_token)

    all_tracks = []

    for album in albums:
        album_tracks = get_album_tracks(album['id'], access_token)
        all_tracks.extend(album_tracks)  # Añadir las canciones a la lista

    # Obtener playlists
    playlists = get_artist_playlists("Intocable", access_token)

    # Obtener géneros
    genres = get_genres(access_token)

    # Obtener mercados
    markets = get_markets(access_token)

    # Guardar datos en archivos CSV
    save_data_to_csv(all_tracks, 'intocable_tracks.csv')
    save_data_to_csv(albums, 'intocable_albums.csv')
    save_data_to_csv(playlists, 'intocable_playlists.csv')
    save_data_to_csv(genres, 'intocable_genres.csv')
    save_data_to_csv(markets, 'intocable_markets.csv')

    print("Datos de Intocable guardados en archivos CSV.")

Error fetching albums: 404, {"error": {"status": 404, "message": "Resource not found" } }
Datos de Intocable guardados en archivos CSV.


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [87]:
from datasets import load_dataset

# Cargar el dataset desde Hugging Face
dataset = load_dataset("TimoH/spotify_sleep_dataset", split="train")

In [92]:
# Assuming the Hugging Face dataset downloaded to 'spotify_sleep_dataset'
from datasets import load_dataset

# Load the dataset using datasets library
dataset = load_dataset("TimoH/spotify_sleep_dataset", split="train")

# Convert the Hugging Face Dataset to a Spark DataFrame
# Note: This requires PySpark to be configured to work with Pandas UDFs
spark_df = spark.createDataFrame(dataset.to_pandas())

# Show the schema and first few records
spark_df.printSchema()
spark_df.show(5)

root
 |-- Unnamed: 0: long (nullable = true)
 |-- TrackName: string (nullable = true)
 |-- TrackID: string (nullable = true)
 |-- SampleURL: string (nullable = true)
 |-- ReleaseYear: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- key: long (nullable = true)
 |-- mode: long (nullable = true)
 |-- duration_ms: long (nullable = true)
 |-- Popularity: double (nullable = true)
 |-- pNum: long (nullable = true)
 |-- playlistID: string (nullable = true)
 |-- label: string (nullable = true)
 |-- userCat: string (nullable = true)
 |-- demoCat: string (nullable = true)
 |-- length: string (nullable = true)
 

In [94]:
from pyspark.sql.functions import avg

# Promedios de características acústicas por año de lanzamiento
acoustic_trends = spark_df.groupBy("ReleaseYear").agg(
    avg("danceability").alias("avg_danceability"),
    avg("energy").alias("avg_energy"),
    avg("acousticness").alias("avg_acousticness")
).orderBy("ReleaseYear")

acoustic_trends.show()


+-----------+-------------------+--------------------+------------------+
|ReleaseYear|   avg_danceability|          avg_energy|  avg_acousticness|
+-----------+-------------------+--------------------+------------------+
|       0000|              0.494|               0.491|             0.546|
| 1899-12-31|                0.4|                 0.1|             0.982|
|       1910|              0.319|               0.109|             0.989|
|       1930|             0.2815|              0.0077|             0.992|
|       1933|              0.358|               0.452|             0.991|
|       1936|              0.444|                0.39|             0.814|
|       1939|              0.342|               0.126|             0.862|
| 1942-01-01|                0.7|               0.104|             0.972|
|       1947|              0.257|               0.168|             0.946|
|       1949|0.46116666666666667|0.023048333333333337|0.9846666666666666|
| 1949-08-01|              0.317|     

In [95]:
# Agrupar por decadas
from pyspark.sql.functions import floor

# Crear columna "Decade" truncando el año a la década
spark_df = spark_df.withColumn("Decade", (floor(spark_df["ReleaseYear"] / 10) * 10))


In [97]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col # Import 'col' here

# Crear ventana para particionar por década y género, ordenar por popularidad
window = Window.partitionBy("Decade", "Genres").orderBy(col("Popularity").desc())

# Obtener la canción más popular de cada género en cada década
top_tracks_by_decade_genre = spark_df.withColumn("rank", row_number().over(window)) \
                                     .filter(col("rank") == 1) \
                                     .select("Decade", "Genres", "TrackName", "Popularity")

top_tracks_by_decade_genre.show()

+------+--------------------+--------------------+----------+
|Decade|              Genres|           TrackName|Popularity|
+------+--------------------+--------------------+----------+
|  NULL|["australian chil...|         Sleepy Time|       0.0|
|  NULL|["australian chil...| Lullaby Sleepy Head|      36.0|
|  NULL|["canadian childr...|Octopus (Slippery...|      40.0|
|  NULL|["canadian childr...|          Sleep Song|      29.0|
|  NULL|["children's folk...|       Slippery Fish|      59.0|
|  NULL|["children's folk...|  Movie In My Pillow|      24.0|
|  NULL|["children's folk...|Count Them As The...|      27.0|
|  NULL|["children's folk...|You Are My Sunshi...|      58.0|
|  NULL|["children's folk...|           Nocturnal|      27.0|
|  NULL| ["children's folk"]|   How to Be a Cloud|      28.0|
|  NULL|["children's musi...|    Hush Little Baby|      38.0|
|  NULL|["children's musi...|How Much Is That ...|      20.0|
|  NULL|["children's musi...|           Goodnight|       0.0|
|  NULL|

In [98]:
# Contar valores nulos en la columna "Decade"
null_decade_count = spark_df.filter(col("Decade").isNull()).count()
print(f"Cantidad de valores nulos en 'Decade': {null_decade_count}")

# Contar el total de registros
total_count = spark_df.count()
print(f"Total de registros: {total_count}")

Cantidad de valores nulos en 'Decade': 221380
Total de registros: 225927


In [99]:
# Contar valores nulos en la columna "ReleaseYear"
null_release_year_count = spark_df.filter(col("ReleaseYear").isNull()).count()
print(f"Cantidad de valores nulos en 'ReleaseYear': {null_release_year_count}")


Cantidad de valores nulos en 'ReleaseYear': 0


In [100]:
# Verificar el tipo de datos de "ReleaseYear"
print(spark_df.select("ReleaseYear").dtypes)


[('ReleaseYear', 'string')]


In [103]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col, when, floor # Import 'floor'

# Convertir "ReleaseYear" a entero, y manejar errores asignando None en caso de valores no numéricos
spark_df = spark_df.withColumn("ReleaseYear", col("ReleaseYear").cast("int"))

# Calcular la columna "Decade" en base a "ReleaseYear"
spark_df = spark_df.withColumn(
    "Decade",
    when((col("ReleaseYear").isNotNull()) &
         (col("ReleaseYear") >= 1900) &
         (col("ReleaseYear") <= 2023),
         floor(col("ReleaseYear") / 10) * 10 # Use floor and division for decade calculation
    ).otherwise(None)
)

# Contar valores nulos en "Decade" después de la corrección
null_decade_count = spark_df.filter(col("Decade").isNull()).count()
print(f"Cantidad de valores nulos en 'Decade' después de la conversión: {null_decade_count}")

Cantidad de valores nulos en 'Decade' después de la conversión: 221381


In [104]:
unique_release_years = spark_df.select("ReleaseYear").distinct().show(truncate=False)


+-----------+
|ReleaseYear|
+-----------+
|1959       |
|1990       |
|1975       |
|1977       |
|2003       |
|2007       |
|2018       |
|1974       |
|2015       |
|1955       |
|2006       |
|1978       |
|1961       |
|2013       |
|1956       |
|1997       |
|1988       |
|1994       |
|1968       |
|2014       |
+-----------+
only showing top 20 rows



In [105]:
invalid_release_years = spark_df.filter((col("ReleaseYear").isNull()) | (col("ReleaseYear") < 1900) | (col("ReleaseYear") > 2023))
invalid_release_years.show()


+----------+--------------------+--------------------+--------------------+-----------+--------------------+------------+------+--------+-----------+------------+----------------+--------+-------+-------+---+----+-----------+----------+----+--------------------+-----+-------+-------+------+-------------+-------+-------+------+
|Unnamed: 0|           TrackName|             TrackID|           SampleURL|ReleaseYear|              Genres|danceability|energy|loudness|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|key|mode|duration_ms|Popularity|pNum|          playlistID|label|userCat|demoCat|length|playlistTitle|  nFoll|nTracks|Decade|
+----------+--------------------+--------------------+--------------------+-----------+--------------------+------------+------+--------+-----------+------------+----------------+--------+-------+-------+---+----+-----------+----------+----+--------------------+-----+-------+-------+------+-------------+-------+-------+------+
|         1| 